In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from climakitae.explore.amy import get_climate_profile

In [ ]:
selection= {
    "variable": "Air Temperature at 2m",
    "resolution": "3 km",
    "warming_level": [2.0],
    "units": "degF",
    # cached_area=['Los Angeles County']
    "latitude": (37.704432 - 0.05, 37.704432 + 0.05),
    "longitude": (-121.898696 - 0.05, -121.898696 + 0.05),
    "q": 0.95,
}

In [ ]:
profile = get_climate_profile(**selection)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

idx = pd.IndexSlice
sims = profile.columns.get_level_values("Simulation").unique().tolist()

for sim in sims:
    # Select columns for the current simulation
    data = profile.loc[:, idx[:, sim]]
    # Assuming the first level is month and the second is simulation
    # We'll plot a heatmap of the data for each simulation
    plt.figure(figsize=(12, 6))
    plt.imshow(data.values.T, aspect='auto', cmap='coolwarm')
    plt.colorbar(label=f"{selection['variable']} ({selection['units']})")
    plt.yticks(range(data.shape[1]), data.columns.get_level_values(0))
    plt.xticks(range(data.shape[0]), data.index)
    plt.title(f"{sim} - {selection['variable']} at {selection['warming_level'][0]}°C Warming")
    plt.xlabel("Day of Year")
    plt.ylabel("Hour of Day")
    vmin = -max(abs(data.values.max()), abs(data.values.min()))
    vmax = max(abs(data.values.max()), abs(data.values.min()))
    xtick_freq = max(1, data.shape[0] // 12)
    plt.xticks(range(0, data.shape[0], xtick_freq), data.index[::xtick_freq])
    plt.clim(vmin, vmax)
    plt.show()



NameError: name 'pd' is not defined